# QAOA Tasks Tutorial

## Problem Generation

First, we generate and save all of the random instances of the problem. This is not computationally intensive but very important to do first so we have a fixed set of random instances.

In [1]:
import recirq
from recirq.qaoa.experiments.problem_generation_tasks import \
    SKProblemGenerationTask, HardwareGridProblemGenerationTask, ThreeRegularProblemGenerationTask, \
    generate_3_regular_problem, generate_sk_problem, generate_hardware_grid_problem

pgen_dataset_id = '2020-03-tutorial'
hardware_grid_problem_tasks = [
    HardwareGridProblemGenerationTask(
        dataset_id=pgen_dataset_id,
        device_name='Sycamore23',
        instance_i=i,
        n_qubits=n
    )
    for i in range(5)
    for n in range(2, 8 + 1, 2)
]
recirq.display_markdown_docstring(HardwareGridProblemGenerationTask)

### HardwareGridProblemGenerationTask
Generate 'Hardware Grid' problems for a named device.

This is a subgraph of the device's harware topology with random
+-1 weights on edges.

#### See Also
`generate_hardware_grid_problem` 

#### Attributes
 - `dataset_id`: A unique identifier for this dataset.
 - `device_name`: The device to generate problems for.
 - `instance_i`: Generate random instances indexed by this number.
 - `n_qubits`: Generate an n-qubit instance.


In [2]:
sk_problem_tasks = [
    SKProblemGenerationTask(
        dataset_id=pgen_dataset_id,
        instance_i=i,
        n_qubits=n
    )
    for i in range(5)
    for n in range(3, 7 + 1, 2)
]
recirq.display_markdown_docstring(SKProblemGenerationTask)

### SKProblemGenerationTask
Generate a Sherrington-Kirkpatrick problem.

This is a complete (fully-connected) graph with random +-1
weights on edges.

#### See Also
`generate_sk_problem` 

#### Attributes
 - `dataset_id`: A unique identifier for this dataset.
 - `instance_i`: Generate random instances indexed by this number.
 - `n_qubits`: Generate an n-qubit instance.


In [3]:
three_regular_problem_tasks = [
    ThreeRegularProblemGenerationTask(
        dataset_id=pgen_dataset_id,
        instance_i=i,
        n_qubits=n
    )
    for i in range(5)
    for n in range(3, 8 + 1) if 3 * n % 2 == 0
]

### Run the tasks

In [ ]:
for task in hardware_grid_problem_tasks:
    generate_hardware_grid_problem(task)
for task in sk_problem_tasks:
    generate_sk_problem(task)
for task in three_regular_problem_tasks:
    generate_3_regular_problem(task)

## Angle Precomputation

In [5]:
from recirq.qaoa.experiments.angle_precomputation_tasks import \
    AnglePrecomputationTask, precompute_angles

apre_dataset_id = '2020-03-tutorial'
precompute_tasks = [
    AnglePrecomputationTask(
        dataset_id=apre_dataset_id,
        generation_task=gen_task,
        p=p)
    for gen_task in recirq.roundrobin(
        hardware_grid_problem_tasks,
        sk_problem_tasks,
        three_regular_problem_tasks,
    )
    for p in range(1, 3 + 1)
]
recirq.display_markdown_docstring(AnglePrecomputationTask)

### AnglePrecomputationTask
Pre-compute optimized angles classically for a given problem.

#### See Also
`precompute_angles` 

#### Attributes
 - `dataset_id`: A unique identifier for this dataset.
 - `generation_task`: The input task which specifies the problem.
 - `p`: QAOA depth hyperparameter p. The number of parameters is 2*p.


In [ ]:
for task in precompute_tasks:
    precompute_angles(task)

## Precomputed Angle Data Collection

In [7]:
from recirq.qaoa.experiments.precomputed_execution_tasks import \
    PrecomputedDataCollectionTask, collect_data
    

dcol_dataset_id = '2020-03-tutorial'
data_collection_tasks = [
    PrecomputedDataCollectionTask(
        dataset_id=dcol_dataset_id,
        precomputation_task=pre_task,
        device_name='Syc23-simulator',
        n_shots=50_000,
        structured=True,
    )
    for pre_task in precompute_tasks
]
recirq.display_markdown_docstring(PrecomputedDataCollectionTask)

### PrecomputedDataCollectionTask
PrecomputedDataCollectionTask(dataset_id: str, precomputation_task: recirq.qaoa.experiments.angle_precomputation_tasks.AnglePrecomputationTask, device_name: str, n_shots: int, structured: bool = False, echoed: bool = False)

In [ ]:
await recirq.execute_in_queue(collect_data, data_collection_tasks, num_workers=2)

## Landscape Data Collection

In [9]:
from recirq.qaoa.experiments.p1_landscape_tasks import \
    P1LandscapeDataCollectionTask, \
    get_data_collection_tasks_on_a_grid, \
    collect_either_landscape_or_cal

recirq.display_markdown_docstring(P1LandscapeDataCollectionTask)

### P1LandscapeDataCollectionTask
Collect data for a p=1 landscape

This task does exactly one (beta, gamma) point. You will have
to run many of these tasks to get a full landscape

#### See Also
`collect_p1_landscape_data` 

#### Attributes
 - `dataset_id`: A unique identifier for this dataset.
 - `generation_Task`: The task specifying the problem to collect data for
 - `device_name`: The device to run on
 - `n_shots`: The number of shots to take
 - `gamma`: The problem unitary parameter gamma
 - `beta`: The driver unitary parameter beta
 - `line_placement_strategy`: Only used for SK model problems. Options include 'brute_force', 'random', 'greedy', 'anneal', 'mst', and 'mixed'.


In [ ]:
hardware_grid_problem_task = HardwareGridProblemGenerationTask(
    dataset_id=pgen_dataset_id,
    device_name='Sycamore23',
    instance_i=0,
    n_qubits=4
)
data_collection_tasks = get_data_collection_tasks_on_a_grid(
    pgen_task=hardware_grid_problem_task,
    dataset_id=dcol_dataset_id,
    gamma_res=11,
    beta_res=11,
    device_name='Syc23-simulator',
    epoch="grid")

await recirq.execute_in_queue(collect_either_landscape_or_cal,
                              data_collection_tasks,
                              num_workers=2)

In [ ]:
sk_problem_task = SKProblemGenerationTask(
    dataset_id=pgen_dataset_id,
    instance_i=0,
    n_qubits=3,
)
data_collection_tasks = get_data_collection_tasks_on_a_grid(
    pgen_task=sk_problem_task,
    dataset_id=dcol_dataset_id,
    gamma_res=11,
    beta_res=11,
    device_name='Syc23-simulator',
    epoch="sk")
await recirq.execute_in_queue(collect_either_landscape_or_cal,
                              data_collection_tasks,
                              num_workers=2)

In [ ]:
three_regular_problem_task = ThreeRegularProblemGenerationTask(
    dataset_id=pgen_dataset_id,
    instance_i=0,
    n_qubits=4
)
data_collection_tasks = get_data_collection_tasks_on_a_grid(
    pgen_task=three_regular_problem_task,
    dataset_id=dcol_dataset_id,
    device_name='Syc23-simulator',
    gamma_res=11,
    beta_res=11,
    epoch="tr")

await recirq.execute_in_queue(collect_either_landscape_or_cal,
                              data_collection_tasks,
                              num_workers=2)